In [1]:
import torch

In [2]:
a = torch.ones(10)

In [3]:
a

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [4]:
a[1]

tensor(1.)

In [5]:
a[9] = 2.0

In [6]:
a[8] = int(8)

In [7]:
a

tensor([1., 1., 1., 1., 1., 1., 1., 1., 8., 2.])

### Named Tensors

In [8]:
# create dummy image
image_dummy = torch.randn(3, 256, 256) # shape=[channels, rows, columns]

# create a dummy image batch
image_dummy_batch = torch.randn(10, 3, 256, 256) # shape=[batch, channels, rows, columns]

In [9]:
# take mean of all the channels to obtain a grayscale image
print(image_dummy.mean(-3))
print(image_dummy.mean(-3).shape) # grayscale image; shape=[columns, rows] 

tensor([[-0.0674, -0.1689, -1.1018,  ..., -0.7322, -0.2939,  0.3841],
        [-0.2840, -0.7412,  0.1809,  ..., -0.1558, -0.2215, -0.2885],
        [-0.8739, -0.0669,  0.1476,  ..., -0.2360,  0.2177, -0.4072],
        ...,
        [-0.0314, -0.6882, -1.7180,  ..., -0.2414, -1.1165,  0.8286],
        [-0.0416,  0.4167,  0.2723,  ...,  0.5075,  0.2172, -0.2410],
        [ 0.0192,  0.5361,  0.0802,  ..., -0.4366,  0.3520, -0.0738]])
torch.Size([256, 256])


In [10]:
# repeat the same for a batch of images
print(image_dummy_batch.mean(-3).shape) # batch of grayscale images; shape=[batch, columns, rows]

torch.Size([10, 256, 256])


In [11]:
# take weighted mean - multiply the channels with their respective weights and then take mean
weights = torch.tensor([0.2126, 0.7152, 0.0722])

In [12]:
# transform weights by adding extra dimensions
# unsqueeze adds extra dimension
unsqueezed_weights = weights.unsqueeze(1).unsqueeze(1)

In [13]:
# broadcast these weights onto the image
image_dummy_weighted = image_dummy * unsqueezed_weights

In [14]:
# broadcast these weights onto batch of Images
image_dummy_batch_weighted = image_dummy_batch * unsqueezed_weights

In [15]:
# get the weighted grayscale images
image_dummy_grayscale_weighted = image_dummy_weighted.sum(-3)
image_dummy_grayscale_weighted_batch = image_dummy_batch_weighted.sum(-3)
print(image_dummy_grayscale_weighted.shape, image_dummy_grayscale_weighted_batch.shape)

torch.Size([256, 256]) torch.Size([10, 256, 256])


##### Using Named Tensors

In [25]:
named_weights = weights.refine_names("channels")
image_dummy_named = image_dummy.refine_names("channels", "columns", "rows")
image_dummy_batch_named = image_dummy_batch.refine_names("batch", "channels", "columns", "rows")
print("named weights", named_weights.shape, named_weights.names)
print("Image dummy named", image_dummy_named.shape, image_dummy_named.names)
print("Image dummy batch named", image_dummy_batch_named.shape, image_dummy_batch_named.names)

named weights torch.Size([3]) ('channels',)
Image dummy named torch.Size([3, 256, 256]) ('channels', 'columns', 'rows')
Image dummy batch named torch.Size([10, 3, 256, 256]) ('batch', 'channels', 'columns', 'rows')


##### Performing Operations

In [26]:
# change weights dimensions to align with image dimensions
weights_aligned = named_weights.align_as(image_dummy_named)
weights_aligned.shape, weights_aligned.names

(torch.Size([3, 1, 1]), ('channels', 'columns', 'rows'))

In [27]:
# perform operations along dimensions wrt name
grayscale_named = (image_dummy_named * weights_aligned).sum('channels')
grayscale_named.shape, grayscale_named.names

(torch.Size([256, 256]), ('columns', 'rows'))

In [30]:
# this means take mean of each column for all columns for all channels
image_dummy_named.mean('columns').shape

torch.Size([3, 256])

In [31]:
# this means take mean of each row for all rows for all channels
image_dummy_named.mean('rows').shape

torch.Size([3, 256])

In [32]:
# this means combine all three channels into one channel by taking the mean of respective values along the channels
image_dummy_named.mean('channels').shape

torch.Size([256, 256])

##### drop names

In [33]:
image_dummy_named = image_dummy_named.rename(None)
image_dummy_named.shape, image_dummy_named.names

(torch.Size([3, 256, 256]), (None, None, None))

### Storage

In [37]:
points = torch.ones(size=[3, 4], dtype=torch.int64)

In [38]:
points

tensor([[1, 1, 1, 1],
        [1, 1, 1, 1],
        [1, 1, 1, 1]])

In [41]:
storage_object = points.storage()

In [42]:
storage_object

 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
[torch.LongStorage of size 12]

In [45]:
storage_object.size()

12

In [46]:
storage_object[2] = 2

In [47]:
points

tensor([[1, 1, 2, 1],
        [1, 1, 1, 1],
        [1, 1, 1, 1]])

In [50]:
points.zero_()

tensor([[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]])

In [51]:
points.storage()

 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
[torch.LongStorage of size 12]

In [53]:
points.shape

torch.Size([3, 4])

In [55]:
points += 1

In [56]:
points

tensor([[1, 1, 1, 1],
        [1, 1, 1, 1],
        [1, 1, 1, 1]])

In [57]:
points.shape

torch.Size([3, 4])

In [58]:
points.storage_offset()

0

In [59]:
points.stride()

(4, 1)

In [60]:
points[1, 1] = 0

In [61]:
points

tensor([[1, 1, 1, 1],
        [1, 0, 1, 1],
        [1, 1, 1, 1]])

In [65]:
i, j = 1, 2

In [66]:
points[i, j] == points.storage()[points.storage_offset() + i*points.stride()[0] + j*points.stride()[1]]

tensor(True)

In [67]:
sub_points = points[2]

In [68]:
sub_points

tensor([1, 1, 1, 1])

In [69]:
sub_points.storage_offset()

8

In [70]:
sub_points.stride()

(1,)

In [73]:
points = torch.Tensor([[1, 2], [3, 4], [5, 6]])

In [76]:
id(points.storage)

1824715206728

In [77]:
points_t = points.t()

In [79]:
id(points_t.storage)

1824750927528

In [80]:
points_t.storage()

 1.0
 2.0
 3.0
 4.0
 5.0
 6.0
[torch.FloatStorage of size 6]

In [81]:
points.storage()

 1.0
 2.0
 3.0
 4.0
 5.0
 6.0
[torch.FloatStorage of size 6]

In [82]:
points.storage()[3] = -10

In [83]:
points.storage()

 1.0
 2.0
 3.0
 -10.0
 5.0
 6.0
[torch.FloatStorage of size 6]

In [84]:
points_t.storage()

 1.0
 2.0
 3.0
 -10.0
 5.0
 6.0
[torch.FloatStorage of size 6]

In [85]:
points_t

tensor([[  1.,   3.,   5.],
        [  2., -10.,   6.]])

In [86]:
points

tensor([[  1.,   2.],
        [  3., -10.],
        [  5.,   6.]])

In [87]:
points.storage()

 1.0
 2.0
 3.0
 -10.0
 5.0
 6.0
[torch.FloatStorage of size 6]

In [96]:
points_t

tensor([[  1.,   3.,   5.],
        [  2., -10.,   6.]])

In [89]:
points_t.storage()

 1.0
 2.0
 3.0
 -10.0
 5.0
 6.0
[torch.FloatStorage of size 6]

In [91]:
points.is_contiguous()

True

In [94]:
points_t_cont = points_t.contiguous()

In [95]:
points_t_cont

tensor([[  1.,   3.,   5.],
        [  2., -10.,   6.]])

In [102]:
points_t_cont.storage()

 1.0
 3.0
 5.0
 2.0
 -10.0
 6.0
[torch.FloatStorage of size 6]

In [103]:
points_t.storage()

 1.0
 2.0
 3.0
 -10.0
 5.0
 6.0
[torch.FloatStorage of size 6]

In [101]:
id(points_t.storage) == id(points_t_cont.storage)

True

In [106]:
points_t.storage

<function Tensor.storage>

In [107]:
x = torch.Tensor([1, 2])

In [110]:
id(x.storage) == id(points_t.storage)

True

In [118]:
id(points_t.storage())

1824772856648

In [119]:
id(x.storage())

1824772855880

In [120]:
id(points_t.storage()) == id(x.storage())

True

In [117]:
x.storage()

 1.0
 2.0
[torch.FloatStorage of size 2]

In [121]:
a = torch.Tensor(list(range(9)))

In [122]:
a

tensor([0., 1., 2., 3., 4., 5., 6., 7., 8.])

In [124]:
b = a.view(3, 3)

In [125]:
a.storage()

 0.0
 1.0
 2.0
 3.0
 4.0
 5.0
 6.0
 7.0
 8.0
[torch.FloatStorage of size 9]

In [126]:
b.storage()

 0.0
 1.0
 2.0
 3.0
 4.0
 5.0
 6.0
 7.0
 8.0
[torch.FloatStorage of size 9]

In [127]:
a.stride()

(1,)

In [128]:
b.stride()

(3, 1)